<a href="https://colab.research.google.com/github/samratapoorv/Ongc-project/blob/main/ongcproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 18.9 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import os
import re
import shutil
import zipfile
import spacy

In [ ]:
# Define the keywords to look for
keywords = [
    "Location", "Geological", "Structure", "Seismic Lines",
    "Log motif", "Well construction diagram", "drilling",
    "Geotechnical order", "Stratigraphy and casing plot",
    "Remote sensing image", "Contour", "Structure Contour map",
]


In [ ]:
# Create folders for each keyword
for keyword in keywords:
    os.makedirs(keyword, exist_ok=True)

In [ ]:
# Function to clean and normalize text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

In [ ]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Function to categorize text using spaCy
def categorize_text_advanced(text):
    text = clean_text(text)
    doc = nlp(text)
    for keyword in keywords:
        if any(keyword.lower() in token.text.lower() for token in doc):
            return keyword
    return None

In [ ]:
# Open the PDF file
pdf_document = fitz.open("/content/well_student.pdf")

In [ ]:
# Define size criteria (in bytes)
size_threshold = 100 * 1024  # Example size threshold: 100KB

In [ ]:
for page_num in range(len(pdf_document)):
    page = pdf_document.load_page(page_num)
    images = page.get_images(full=True)

    # Extract text from the page
    text = page.get_text("text")

    for img_index, img in enumerate(images):
        xref = img[0]
        base_image = pdf_document.extract_image(xref)
        image_bytes = base_image["image"]

        # Check if the image size meets the criteria
        if len(image_bytes) >= size_threshold:
            # Categorize the image based on the surrounding text
            category = categorize_text_advanced(text)
            if category:
                image_filename = f"{category}/page_{page_num + 1}_image_{img_index + 1}.png"
                with open(image_filename, "wb") as img_file:
                    img_file.write(image_bytes)
            else:
                # If no category found, put in 'Uncategorized' folder
                os.makedirs("Uncategorized", exist_ok=True)
                image_filename = f"Uncategorized/page_{page_num + 1}_image_{img_index + 1}.png"
                with open(image_filename, "wb") as img_file:
                    img_file.write(image_bytes)

In [ ]:
# Zip each folder
def zip_folder(folder_name):
    shutil.make_archive(folder_name, 'zip', folder_name)

for keyword in keywords:
    zip_folder(keyword)

zip_folder("Uncategorized")

# Output the path of the zip files for downloading
zipped_files = [f"{keyword}.zip" for keyword in keywords] + ["Uncategorized.zip"]
print(zipped_files)